In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

In [ ]:
data = pd.read_excel("/kaggle/input/berts-orj/clean_orj_ttc.xlsx")

data.columns = ["text", "label"]

data.head()

In [ ]:
data["labels"] = pd.factorize(data.label)[0]

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train=train[["text","labels"]]
test=test[["text","labels"]]

In [ ]:
train["text"]=train["text"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [ ]:
train.shape,test.shape

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 10

# cuda için gpu seçili olması gerekir.
model = ClassificationModel('bert', 'bert-base-cased', num_labels=6, use_cuda=True, args=model_args)

In [ ]:
model.train_model(train)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

predictions = model_outputs.argmax(axis=1)
 
actuals = test.labels.values

from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(actuals, predictions)

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,12))
plt.figure(facecolor='#ffffff')
g = sns.heatmap(cm,cmap="Blues", fmt="", annot=True)
g.set_xticklabels(data.label.unique(), fontsize = 12, rotation = 75)
g.set_yticklabels(data.label.unique(), fontsize = 12, rotation = 0)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
clr = classification_report(actuals, predictions, target_names=data.label.unique(), digits= 4) # create classification report
print("Classification Report:\n----------------------\n", clr)